In [1]:
import os
import torch
import torchvision as tv
import h5py
import numpy as np
import torch
import fastmri
import random
import sys
import torchvision
import pytorch_lightning as pl

sys.path.append('../../src/')
from helpers import calculate_metrics

from pathlib import Path
from torch import nn
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils import data
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# try:
#     print(torch.cuda.get_device_name())
#     print(torch.cuda.get_device_properties(0))
# except:
#     print("No GPUs configured")
    
manualSeed = 42
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  42


In [5]:
import fastmri_run_pretrained_varnet_inference 


challenge = "varnet_knee_mc"
state_dict_file = None
data_path = Path("")

fastmri_run_pretrained_varnet_inference.run_inference(
    challenge,
    state_dict_file,
    data_path,
    output_path
)

In [2]:
from fastmri.data.mri_data import fetch_dir

path_config = Path("../../fastmri_dirs.yaml")

dataroot = fetch_dir("knee_path", path_config)
test_path = Path("../../mnt/FastMRI/")
dataroot

PosixPath('/mri-reconstruction/mnt/FastMRI/knee')

In [3]:
import os

for i in dataroot.glob("**/*.h5"):
    try:
        hf = h5py.File(i, mode='r')
    except:
        print("Truncated: ", i)
#         os.remove("{}".format(i))

# PyTorch Dataset

In [4]:
from fastmri.pl_modules import FastMriDataModule, VarNetModule
from fastmri.data.transforms import VarNetDataTransform
from fastmri.data.subsample import create_mask_for_mask_type

# singlecoil or multicoil
challenge = "singlecoil"
# MASK_TYPE is either random (for knee) or equispaced (for brain)
mask_type = "random"
center_fractions = [0.08]
accelerations = [4]

mask = create_mask_for_mask_type(
    mask_type, center_fractions, accelerations
)

# use random masks for train transform, fixed masks for val transform
train_transform = VarNetDataTransform(mask_func=mask, use_seed=False)
val_transform = VarNetDataTransform(mask_func=mask)
test_transform = VarNetDataTransform()
    
data_module = FastMriDataModule(
    data_path=dataroot,
    challenge=challenge,
    train_transform=train_transform,
    val_transform=val_transform,
    test_transform=test_transform,
    test_path=test_path
)

# VarNet Model

In [5]:
model = VarNetModule(
    num_cascades=8,  # number of unrolled iterations
    pools=4,  # number of pooling layers for U-Net
    chans=18,  # number of top-level channels for U-Net
    sens_pools=4,  # number of pooling layers for sense est. U-Net
    sens_chans=8,  # number of top-level channels for sense est. U-Net
    lr=0.001,  # Adam learning rate
    lr_step_size=40,  # epoch at which to decrease learning rate
    lr_gamma=0.1,  # extent to which to decrease learning rate
    weight_decay=0.0,  # weight regularization strength
)

# Train model

In [6]:
trainer = pl.Trainer()
trainer.fit(model, datamodule=data_module)

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
GPU available: False, used: False
TPU available: None, using: 0 TPU cores


OSError: Unable to open file (file signature not found)